# STEP 1

In [1]:
# importo le librerie

import shutil
import os
import csv
import pandas as pd
import numpy as np
from tabulate import tabulate
from PIL import Image

In [2]:
print(os.getcwd())
"""
a che serve questa cella?

per essere il più generico possibile, puoi:
- trovare il percorso della cartella corrente (os.getcwd)
- da lì, riferirti a (o spostarti in: os.path.chdir) quella di lavoro, files/
"""

c:\Users\Edoardo\machine learning\progetti start2impact\file organizer


'\na che serve questa cella?\n\nper essere il più generico possibile, puoi:\n- trovare il percorso della cartella corrente (os.getcwd)\n- da lì, riferirti a (o spostarti in: os.path.chdir) quella di lavoro, files/\n'

In [3]:
os.chdir('files') # sposto la directory di lavoro in files
    
"""
attenzione alla gestione del file di recap:

il tuo codice lo riscrive (da capo, con l'header) ogni volta che viene lanciato lo script;
dovresti pensare a una logica che
- se il file di recap NON ESISTE, viene creato (con l'header, che è sempre la prima riga)
e viene riempito con le info dei file spostati
- se ESISTE, viene solo riempito (AGGIORNANDO la versione già esistente)

pensa inoltre a quali di queste operazioni conviene mettere nel ciclo principale (e quindi
eseguire ogni volta che viene spostato un file) e cosa puoi esternalizzare (e
quindi eseguire una volta sola)
per evitare (*) di aprire e chiudere il file di recap ad ogni iterazione del ciclo sui file
da spostare, come hai fatto qui sotto, puoi:
- aprirlo prima e chiuderlo alla fine
- mettere tutto il ciclo principale nel corpo di un "with"

"""
    
files_list = [] # creo una lista in cui inserire le informazioni dei file
img_formats = ['png', 'jpg', 'jpeg'] # creo lista per formato immagini
txt_formats = ['txt', 'odt'] # creo lista per formato testi

if not os.path.isdir('images'): # se non è presente, creo la cartella 'images'
    os.mkdir('images')

if not os.path.isdir('audio'): # se non è presente, creo cartella per file audio
    os.mkdir('audio')
    
if not os.path.isdir('doc'): # se non è presente, creo cartella per file testuali
    os.mkdir('doc')


if not os.path.isfile('recap.csv'): # se il recap non è presente, lo creo
    
    recap = open('recap.csv', mode = 'w')
    writer = csv.writer(recap)
    writer.writerow(['filename', 'type', 'size'])
    
else: # se il recap è già presente lo aggiorno
    
    recap = open('recap.csv', mode = 'a') 
    writer = csv.writer(recap)
    
for file in os.listdir(os.curdir): # itero sugli elementi della cartella files

    if os.path.isfile(file): # seleziono solo i file

        filepath = os.path.abspath(file) # assegno a filepath il percorso del file
        size = os.path.getsize(filepath) # assegno a size la dimensione
        filename, ext = file.split('.') # assegno a filename il nome e a ext il formato del file

        """
        oltre a inserire queste info nel recap, dovresti stamparle a video
        durante l'esecuzione dello script
        """

        if ext == 'csv': # se il file è .csv (recap) riparto dall'inizio del ciclo 
            continue

        elif ext in img_formats: # seleziono le immagini

            shutil.move(filepath, 'images') # sposto l'immagine in images
            file_type = 'image' # assegno a file_type il tipo del file

        # ripeto il processo con i file audio e testuali

        elif ext == 'mp3':
            shutil.move(filepath, 'audio')
            file_type = 'audio'

        elif ext in txt_formats:

            shutil.move(filepath, 'doc')
            file_type = 'doc'
            
        # segnalo file non supportati

        else:
            print(f'I file di tipo {ext} non sono supportati!')

        files_list.append(f'{filename}  size: {size}B  type: {file_type}') # inserisco le info del file in files_list

        data = pd.read_csv('recap.csv')
        
        filenames = data.filename.tolist()
        
        # se è già presente nel recap un file con lo stesso nome, lo aggiungo solo se dimensione e formato sono diversi

        if filename in filenames:
            
            if file_type != data.type[filenames.index(filename)] and f'{size}B' != data['size'][filenames.index(filename)]:
                    
                    writer.writerow([filename, file_type, f'{size}B'])
            
            else:
                
                pass
                    
        else: # se non è presente un file con quel nome, lo aggiungo al recap
            
            writer.writerow([filename, file_type, f'{size}B'])
            
recap.close() # chiudo il file di recap

p = [print(file) for file in sorted(files_list)] # stampo la lista dei file in ordine alfabetico

os.chdir('..') # ripristino la directory originale

In [5]:
nd = pd.read_csv('.//files//recap.csv') # controllo il contenuto di recap
print(nd)

  filename   type      size
0    song1  audio  1087849B


In [ ]:
"""
step 2

bene
- l'utilizzo di argparse
- l'adattamento dello step 1 al caso "singolo file"
- l'interazione con l'utente

(vale tutto quello che ti ho detto per lo step 1)

una funzione "main" (che contiene il corpo chiamante del codice)
non è sbagliata, ma neanche necessaria;
volendo, puoi lasciare "procedurale" questa parte del codice, cioè fuori
dal corpo di una funzione

hai capito a che serve il costrutto if __name__ == '__main__'?

https://www.programmareinpython.it/video-corso-python-intermedio/03-if-name-main/
https://www.freecodecamp.org/news/if-name-main-python-example/
"""

---

# STEP 3

In [33]:
os.chdir('files') # sposto la directory per essere più agevolato

table = []

for image in os.listdir('images'): # itero sulle immagini nella cartella
    
    img = Image.open(os.path.join('images', image)) # apro l'immagine
    np_img = np.array(img) # creo un array a partire dall'immagine
    
    if np_img.ndim == 2: # se l'array ha solo due dimensioni, l'immagine è in scala grigi
        #creo una lista con le caratteristiche delle immagini
        row = [image.split('.')[0], np_img.shape[0], np_img.shape[1], np_img.mean(), 0., 0., 0., 0.]
        """
        puoi scrivere 0. oppure .0 oppure 0.0 invece che float(0)
        """
        table.append(row) #inserisco la lista nella lista 'table'
        #ripeto il processo per le immagini RGB e RGBA

    elif np_img.shape[2] == 3: # se la profondità della terza dimensione è 3, l'immagine è RGB
        red, green, blue = np_img.mean(axis = (0, 1))
        row = [image.split('.')[0], np_img.shape[0], np_img.shape[1], 0., red, green, blue, 0.]
        table.append(row)        
    # """
    # non c'è bisogno di calcolare la media per ogni livello di colore; puoi usare (come?)
    # l'argomento "axis" della funzione mean (che esiste per ogni funzione vettoriale
    # di NumPy)
    # """
        
    elif np_img.shape[2] == 4: # se la profondità della terza dimenzione è 4, l'immagine è in formato RGBA
        red, green, blue, alpha = np_img.mean(axis = (0, 1))
        row = [image.split('.')[0], np_img.shape[0], np_img.shape[1], 0., red, green, blue, alpha]
        table.append(row)
        
    # """
    # ...else? se non ci sono altri casi possibili, puoi chiudere con un else; se invece ci sono,
    # meglio gestirli: chiediti sempre che succede se non vengono eseguiti né il corpo dell'if
    # né quello degli elif; il codice che c'è dopo gira lo stesso?
    # """
    else:
        print('Il formato dell\'immagine non è supportato!')
    
new_table = tabulate(table, headers = ['name', 'height', 'width', 'grayscale', 'R', 'G', 'B', 'ALPHA'], tablefmt = 'fancy_grid', numalign = 'right', floatfmt = '.2f') # creo la tabella utilizzando le caratteristiche dell'immagine come titoli

print(new_table) 

os.chdir('..') # ripristino la directory precendente

╒══════════╤══════════╤═════════╤═════════════╤════════╤════════╤═══════╤═════════╕
│ name     │   height │   width │   grayscale │      R │      G │     B │   ALPHA │
╞══════════╪══════════╪═════════╪═════════════╪════════╪════════╪═══════╪═════════╡
│ bw       │      512 │     512 │       21.48 │   0.00 │   0.00 │  0.00 │    0.00 │
├──────────┼──────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ daffodil │      500 │     335 │        0.00 │ 109.25 │  85.56 │  4.97 │    0.00 │
├──────────┼──────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ eclipse  │      256 │     256 │        0.00 │ 109.05 │ 109.52 │ 39.85 │  133.59 │
├──────────┼──────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ trump    │      183 │     275 │        0.00 │  97.01 │  98.99 │ 90.92 │    0.00 │
╘══════════╧══════════╧═════════╧═════════════╧════════╧════════╧═══════╧═════════╛
